In [ ]:

import os
import sys
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
import jpholiday

from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from scipy import stats
from statsmodels.stats.multitest import multipletests

import warnings
warnings.filterwarnings("ignore")

# 作業ディレクトリの設定
try:
    os.chdir('H:/マイドライブ/03_code_test/clustering-house_trial')
    data_path_header = 'G:/マイドライブ/01_研究/02_円山町/1_データ前処理'
except FileNotFoundError:
    os.chdir('G:/マイドライブ/03_code_test/clustering-house_trial')
    data_path_header = 'H:/マイドライブ/01_研究/02_円山町/1_データ前処理'
print("Current Working Directory: ", os.getcwd())


# カスタムライブラリのパスを追加
sys.path.append(data_path_header)
from column_translation import column_translation_dict

col_name_dict = {
    "electric": "ED",
    "LD": "LD",
    "kitchen": "KT",
    "bedroom": "bed",
    "bathroom": "bath",
    "washing": "WM",
    "dishwasher": "DW"
}

def get_household_size(house_num):
    num_household_dict = {
        80	: 3,
        81	: 6,
        82	: 3,
        83	: 4,
        115	: 3,
        117	: 4,
        118	: 4,
        120	: 3,
        121	: 2,
        124	: 4,
        125	: 4,
        126	: 3,
        127	: 3,
        147	: 4,
        148	: 4,
        150	: 4,
        152	: 6,
        155	: 5,
        156	: 3,
        157	: 2,
        84	: 4,
        92	: 4,
        94	: 4,
        116	: 4,
        119	: 4,
        149	: 2,
        154	: 4,
        158	: 3,
        160	: 3,
        171	: 3,
        172	: 4,
    }
    return num_household_dict.get(house_num, None)

In [ ]:
dff = pd.read_csv('./output_feature/80_LD_energy_metrics.csv')

dff.columns

In [ ]:
features_list = ['month', 'weekday_weekend_ratio',
                 'act-pro_0-6', 'act-pro_6-12', 'act-pro_12-18', 'act-pro_18-24',
                 'time_bin_0-6', 'time_bin_6-12', 'time_bin_12-18', 'time_bin_18-24']

csv_path_list = glob.glob('./output_feature/*.csv')

# ファイル名の先頭2文字を取り出して集合化
prefixes = {int(os.path.basename(p).split('_')[0]) for p in csv_path_list}
house_num_list = sorted(prefixes)
print(house_num_list, len(house_num_list))

df_data = pd.DataFrame()
for house_num in house_num_list:
    csv_path_list = glob.glob(f'./output_feature/{house_num}_*.csv')
    csv_path_list.sort()
    df_data_house = pd.DataFrame()
    for csv_path in csv_path_list:
        df_temp = pd.read_csv(csv_path)
        df_temp = df_temp[features_list]
        col_name = os.path.basename(csv_path).split('_')[1].replace('.csv', '')
        col_name = col_name_dict.get(col_name, col_name)
        df_temp.columns = [f'{col_name}_{col}' if col != 'month' else col for col in df_temp.columns]
        df_data_house = pd.merge(df_data_house, df_temp, on='month') if not df_data_house.empty else df_temp
    df_data_house['household_size'] = get_household_size(house_num)
    df_data_house['house_id'] = house_num
    df_data = pd.concat([df_data, df_data_house], axis=0).reset_index(drop=True)

# print(df_data.head(5))
print(df_data.shape)

# 欠損処理
df_dropped = df_data.drop(['month', 'house_id'], axis=1)
df_dropped = df_dropped.replace([np.inf, -np.inf], np.nan).astype("float64")
imp = SimpleImputer(strategy="median")
X = imp.fit_transform(df_dropped)
clean_imputed = pd.DataFrame(X, columns=df_dropped.columns, index=df_dropped.index)
df_features = clean_imputed
# 正規化
mm = preprocessing.MinMaxScaler()
df_features_mm = pd.DataFrame(mm.fit_transform(df_features), columns=df_features.columns)

# print(df_features_mm.head(5))
print(df_features_mm.shape)


In [ ]:
if df_features_mm.shape[0] / df_features_mm.shape[1] < 2:
    print("サンプル数が少ないため、PCAで次元削減を実施")
    # PCAで次元削減
    n_pca_components = df_features_mm.shape[0]/2.5  # サンプル数や特徴量数に応じて調整
    pca = PCA(n_components=n_pca_components, random_state=42)
    df_features_pca = pca.fit_transform(df_features_mm)
    df_features_mm = df_features_pca
else:
    print("サンプル数が十分にあるため、次元削減は実施しない")

# =========================
# エルボー法
inertia = []
K_range = range(1, 27)  # 1〜14クラスタで確認
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_features_mm)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(K_range, inertia, marker='o')
plt.xlabel("Number of clusters")
plt.ylabel("Inertia (SSE)")
plt.title("Elbow Method")
plt.grid(True)

# =========================
# シルエット分析
sil_scores = []
K_range_sil = range(2, 27)  # シルエットスコアはk>=2
for k in K_range_sil:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_features_mm)
    score = silhouette_score(df_features_mm, labels)
    sil_scores.append(score)

plt.subplot(1, 2, 2)
plt.plot(K_range_sil, sil_scores, marker='o')
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Analysis")
plt.grid(True)

plt.tight_layout()
plt.show()

# =========================
# 推奨kの候補表示
best_sil_idx = sil_scores.index(max(sil_scores))
recommended_k = K_range_sil[best_sil_idx]
print(f"シルエットスコア最大のkの候補: {recommended_k}")

In [ ]:
print('---kmeans clusters---')
k = 8 # クラスター数を指示
kmeanModel = KMeans(n_clusters=k, random_state=42)
kmeanModel.fit(df_features_mm)
clusters_kmeans = kmeanModel.labels_
# クラスターごとに何サンプルあるか
for i in range(k):
    num = list(clusters_kmeans).count(i)
    print(f'Cluster {i}: n = {num}')

list_clusters_kmeans = list(set(clusters_kmeans))
colors_kmeans = plt.cm.get_cmap("hsv", len(list_clusters_kmeans))
print(list_clusters_kmeans, len(list_clusters_kmeans))

print('---house clusters---')
clusters_str = df_data['house_id'].astype(str)
le = LabelEncoder()
clusters_house = le.fit_transform(clusters_str)
list_clusters_house = list(set(le.classes_))
colors_house = plt.cm.get_cmap("hsv", len(list_clusters_house))
print(list_clusters_house, len(list_clusters_house))

print('---month clusters---')
clusters_str = df_data['month'].astype(str)
le = LabelEncoder()
clusters_month = le.fit_transform(clusters_str)
list_clusters_month = list(set(le.classes_))
colors_month = plt.cm.get_cmap("hsv", len(list_clusters_month))
print(list_clusters_month, len(list_clusters_month))


In [ ]:
pca = PCA(random_state=42)
pca.fit(df_features_mm)
score = pd.DataFrame(pca.transform(df_features_mm), index=df_features.index)

plt.scatter(score.iloc[:,0], score.iloc[:,1],
            c=clusters_kmeans, cmap=colors_kmeans, alpha=0.7)
plt.title('PCA plot(kmeans)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar()
plt.show()

plt.scatter(score.iloc[:,0], score.iloc[:,1],
            c=clusters_house, cmap=colors_house, alpha=0.7)
plt.title('PCA plot(house)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar()
plt.show()

plt.scatter(score.iloc[:,0], score.iloc[:,1],
            c=clusters_month, cmap=colors_month, alpha=0.7)
plt.title('PCA plot(month)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar()
plt.show()

In [ ]:

embedding = TSNE(random_state=42).fit_transform(df_features_mm)

plt.scatter(embedding[:, 0], embedding[:, 1],
    c=clusters_kmeans, cmap=colors_kmeans, alpha=0.7)
plt.title('t-SNE plot(kmeans)')
plt.colorbar()
plt.show()

plt.scatter(embedding[:, 0], embedding[:, 1],
    c=clusters_house, cmap=colors_house, alpha=0.7)
plt.title('t-SNE plot(house)')
plt.colorbar()
plt.show()

plt.scatter(embedding[:, 0], embedding[:, 1],
    c=clusters_month, cmap=colors_month, alpha=0.7)
plt.title('t-SNE plot(month)')
plt.colorbar()
plt.show()

In [ ]:

mapper = umap.UMAP(random_state=42)
embedding = mapper.fit_transform(df_features_mm)

plt.scatter(embedding[:, 0], embedding[:, 1],
    c=clusters_kmeans, cmap=colors_kmeans, alpha=0.7)
plt.title('UMAP plot(kmeans)')
plt.colorbar()
plt.show()

plt.scatter(embedding[:, 0], embedding[:, 1],
    c=clusters_house, cmap=colors_house, alpha=0.7)
plt.title('UMAP plot(house)')
plt.colorbar()
plt.show()

plt.scatter(embedding[:, 0], embedding[:, 1],
    c=clusters_month, cmap=colors_month, alpha=0.7)
plt.title('UMAP plot(month)')
plt.colorbar()
plt.show()

In [ ]:
N = len(df_features.columns)  # 実データの個数に置き換え
cols = 5
rows = math.ceil(N / cols)

fig = plt.figure(figsize=(3*cols,3*rows))
features_cols = df_features.columns

for i, col in enumerate(features_cols):
    ax = fig.add_subplot(rows, cols, i+1, title=col)
    ax.scatter(embedding[:, 0], embedding[:, 1],
        c=df_features[col], cmap='plasma', alpha=0.8)
fig.tight_layout()
plt.show()

In [ ]:


clusters = clusters_kmeans
list_clusters = list_clusters_kmeans

df_features_with_clusters = df_features.copy()
df_features_with_clusters['cluster'] = clusters
features_cols = df_features.columns

# 閾値を定める。ここでは補正後のp値 (q値) が0.05以下かつ数値の比 (Fold Change:fc)が2倍以上あるか、を閾値とします。
q_threshold = 0.05
fc_threshold = 2

import math
N = len(set(clusters))
cols = 3
rows = math.ceil(N / cols)
fig = plt.figure(figsize=(4*cols, 4*rows))
# クラスターごとに評価する
for i in range(len(set(clusters))):
    p_values = []
    fcs = []
    # 変数 = プロット上の1点
    for col in features_cols:
        # 検定
        group_1 = df_features_with_clusters[df_features_with_clusters['cluster'] == i][col]
        group_2 = df_features_with_clusters[df_features_with_clusters['cluster'] != i][col]
        p_value = stats.ttest_ind(group_1, group_2, equal_var=False)[1]
        p_values.append(p_value)

        # Fold change. 平均での比較が不適切であればここをmedian等に変える
        fc = group_1.mean()/group_2.mean()
        fcs.append(fc)

    # p-valueの補正
    q_values = multipletests(p_values, method='fdr_bh')[1]

    # 閾値を超えたものは色を変える
    colors = []
    for col, q_value, fc in zip(features_cols, q_values, fcs):
        # 対象がその他の2倍大きいときはオレンジ
        if q_value < q_threshold and fc > fc_threshold:
            colors.append('orange')
        # その他が対象の2倍大きいときは水色
        elif q_value < q_threshold and fc < 1/fc_threshold:
            colors.append('skyblue')
        # 大きな違いがない場合は灰色
        else:
            colors.append('gray')

    ax = fig.add_subplot(rows, cols, i+1)
    ax.scatter(np.log2(fcs), -np.log10(q_values),
    c=colors)

    # 図をきれいに見せるためのあれこれ。好みの世界
    max_val = max(abs(np.nanmin(np.log2(fcs)[np.log2(fcs) != -np.inf])), max(np.log2(fcs)))
    ax.set_xlim([-max_val-1, max_val+1]) # -infがあるので。-inf = そのクラスターでは全員が0
    ax.set_ylim(ax.get_ylim())
    # 閾値に点線をつける
    ax.hlines([-np.log10(q_threshold)], -max_val-1, max_val+1, 'gray', 'dashed', linewidth=0.5, alpha=0.5)
    ax.vlines([np.log2(fc_threshold), np.log2(1/fc_threshold)], ax.get_ylim()[0], ax.get_ylim()[1], 'gray', 'dashed', linewidth=0.5, alpha=0.5)

    # ラベルとアノテーション
    ax.set_title(f'{list_clusters[i]}')
    ax.set_xlabel('logFC')
    ax.set_ylabel('-log10(q-values)')
    for j, label in enumerate(features_cols):
        if colors[j] in ['orange', 'skyblue']:
            ax.annotate(label, (np.log2(fcs)[j], -np.log10(q_values)[j]), size=9)


# fig.suptitle('Volcano plots')
fig.tight_layout()
plt.show()